In [55]:
import tensorflow as tf

from tensorflow.keras.layers import LSTM

# from tensorflow.keras.models import Sequential

from tensorflow.keras.losses import MeanSquaredError
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [56]:
true_path = "C:\\Users\\Daniel\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\CSV_Files\\StbdTesting_1404_002_FINAL.csv"
#training_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\train1_port_new.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\StbdTesting_1404_002.csv"
#test_path = "D:\OneDrive\OneDrive - University of Massachusetts Dartmouth\ECE457_Senior_Design_ECE5\CSV_Files\\test1_stbd_new.csv"
prediction_path = "C:\\Users\\Daniel\\OneDrive - University of Massachusetts Dartmouth\\ECE457_Senior_Design_ECE5\\AIOutput_CSV_Files\\DL_PredictedOutputTestThree3_12_2022.csv"

In [57]:
df_true=pd.read_csv(true_path)
#df_test=pd.read_csv(test_path)
df_prediction=pd.read_csv(prediction_path)


In [58]:

#df_predictedDOA = df_prediction.iloc[:,4]
#df_prediction
df_predictedDOA = df_prediction[["PredictedDOA"]]
df_predictedDOA

,PredictedDOA
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
5480113,82.290581
5480114,82.336884
5480115,82.382286
5480116,82.889717


In [59]:
df_reference = df_true[["DOA", "AngleUncertainty"]]
df_reference

,DOA,AngleUncertainty
0,55.834999,180
1,55.834999,191
2,55.834999,255
3,55.834999,255
4,70.369998,190
...,...,...
5480113,82.629998,68
5480114,80.584998,38
5480115,79.484998,93
5480116,78.554998,123


In [60]:
df_reference['AngleUncertainty'].max()

255

In [61]:
df_reference['AngleUncertainty'].mean()

104.45736241445896

In [62]:
df_reference['AngleUncertainty'].var()

6889.18303365501

In [63]:
df_comparison = df_predictedDOA.join(df_reference)
df_comparison

,PredictedDOA,DOA,AngleUncertainty
0,0.000000,55.834999,180
1,0.000000,55.834999,191
2,0.000000,55.834999,255
3,0.000000,55.834999,255
4,0.000000,70.369998,190
...,...,...,...
5480113,82.290581,82.629998,68
5480114,82.336884,80.584998,38
5480115,82.382286,79.484998,93
5480116,82.889717,78.554998,123


In [64]:
theShape = df_comparison.shape
numRows= theShape[0]

In [65]:
numCols = 1
#5504457 
df_results = pd.DataFrame(index=range(numRows),columns=range(numCols))
#df_results = pd.DataFrame({'PassFail'})
#df_comparison.insert()
df_results.columns = {"PassFail"}
df_results

,PassFail
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
5480113,NaN
5480114,NaN
5480115,NaN
5480116,NaN


In [66]:
df_comparison = df_comparison.join(df_results)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail
0,0.000000,55.834999,180,NaN
1,0.000000,55.834999,191,NaN
2,0.000000,55.834999,255,NaN
3,0.000000,55.834999,255,NaN
4,0.000000,70.369998,190,NaN
...,...,...,...,...
5480113,82.290581,82.629998,68,NaN
5480114,82.336884,80.584998,38,NaN
5480115,82.382286,79.484998,93,NaN
5480116,82.889717,78.554998,123,NaN


In [67]:
df_comparison['Delta'] = df_comparison['PredictedDOA'] - df_comparison['DOA']
df_comparison['AbsDelta'] = df_comparison['Delta'].abs()
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,55.834999,180,NaN,-55.834999,55.834999
1,0.000000,55.834999,191,NaN,-55.834999,55.834999
2,0.000000,55.834999,255,NaN,-55.834999,55.834999
3,0.000000,55.834999,255,NaN,-55.834999,55.834999
4,0.000000,70.369998,190,NaN,-70.369998,70.369998
...,...,...,...,...,...,...
5480113,82.290581,82.629998,68,NaN,-0.339417,0.339417
5480114,82.336884,80.584998,38,NaN,1.751885,1.751885
5480115,82.382286,79.484998,93,NaN,2.897288,2.897288
5480116,82.889717,78.554998,123,NaN,4.334719,4.334719


In [68]:
# less than (0.02*AngleUncertainty)/2, since this is the 2sigma value, and we want all values to be under 1 sigma.
df_comparison['PassFail'] = df_comparison['AbsDelta'] <= (0.02*df_comparison['AngleUncertainty']/2)
df_comparison

,PredictedDOA,DOA,AngleUncertainty,PassFail,Delta,AbsDelta
0,0.000000,55.834999,180,False,-55.834999,55.834999
1,0.000000,55.834999,191,False,-55.834999,55.834999
2,0.000000,55.834999,255,False,-55.834999,55.834999
3,0.000000,55.834999,255,False,-55.834999,55.834999
4,0.000000,70.369998,190,False,-70.369998,70.369998
...,...,...,...,...,...,...
5480113,82.290581,82.629998,68,True,-0.339417,0.339417
5480114,82.336884,80.584998,38,False,1.751885,1.751885
5480115,82.382286,79.484998,93,False,2.897288,2.897288
5480116,82.889717,78.554998,123,False,4.334719,4.334719


In [69]:
PassCount = df_comparison['PassFail'].sum()
PassCount

742994

In [70]:
FailCount = (~df_comparison['PassFail']).sum()
FailCount

4737124

In [71]:
TotalCount = PassCount+FailCount
TotalCount

5480118

In [72]:
PercentPass = 100*(PassCount/TotalCount)
PercentPass

13.557992729353638